# Fisheries Training Part 2 - Visualizing Optimization Output

This is the third post in a training series, studying decision making under deep uncertainty within the context of a complex harvested predator-prey fishery. 

The first post in this training series, [Fisheries Training 0: Exploring Predator-Prey Dynamics, can be found here](https://waterprogramming.wordpress.com/2022/07/11/__trashed-3/).

The second post in this training series, [Fisheries Training Part 1 – Harvest Optimization and MOEA Diagnostics](https://waterprogramming.wordpress.com/2022/08/08/fisheries-training-part-1-harvest-optimization-and-moea-diagnostics/). 

The GitHub repository, containing all of the source code used throughout this series, is available [here](https://github.com/TrevorJA/harvested_predator_prey_system_tutorial_series). 

## A recap on our harvest strategy objectives 

#### Objective 1: Net present value

The NPV ($O_1$) is calculated as:
$$O_1 = \frac{1}{N} \sum_{i=1}^N\Big( \sum_{t=0}^T \frac{z_{t+1,i}x_{t,i}}{(1+\delta)^t}\Big)$$

where $\delta$ is the discount rate which converts future benefits to present economic value, here $\delta = 0.05$. 

<br />

#### Objective 2: Prey population deficit

$$O_2 = \frac{1}{N} \sum_{i=1}^N\Big( \frac{1}{T} \sum_{t=1}^T \frac{K - x_{t,i}}{K}\Big)$$

where $K$ is the prey population carrying capacity.

<br />

#### Objective 3: Longest duration of consecutive low harvest

$$O_3 = \frac{1}{N} \sum_{i=1}^N(max_T(\phi_{t,i}))$$

where 

$$
\phi_{t,i} = \left\{
	\begin{array}\\
		\phi_{t-1,i} + 1 & \text{if } \ z_t < \text{limit}\\
		0 & \text{otherwise.}
	\end{array}
\right.
$$

<br />

#### Objective 4: Worst harvest instance


$$O_4 = \frac{1}{N} \sum_{i=1}^N(percentile_T(z_{t+1,i}x_{t,i}, 1))$$

<br />

#### Objective #5: Harvest variance


$$O_5 = \frac{1}{N} \sum_{i=1}^N(Var_T(z_{t+1,i}x_{t,i}))$$

<br />

#### Constraint: Avoid collapse of predator population


$$\frac{1}{N} \sum_{i=1}^N(\Psi_{t,i})) = $$

where 

$$
\Psi_{t,i} = \left\{
	\begin{array}\\
		1 & \text{if } \ y_{t,i} < 1\\
		0 & \text{otherwise.}
	\end{array}
\right.
$$

<br />

#### Problem formulation

$$Minimize \ F(z_x) = (-O_1, O_2, O_3, -O_4, O_5)$$


## Optimizing the Fisheries Game and storing the outcomes
### Import all libraries
All functions required for this post can be found in the `fish_game_functions.py` file. This code is adapted from [Antonia Hadjimichael's original post](https://waterprogramming.wordpress.com/2021/08/13/introduction-to-pyborg-basic-setup-and-running/) on exploring the Fisheries Game dynamics using PyBorg.

In [6]:
# import all required libraries
from platypus import Problem, Real, Hypervolume, Generator
from pyborg import BorgMOEA
from fish_game_functions import *
from platypus import Problem, Real, Hypervolume, Generator
from pyborg import BorgMOEA
import matplotlib.pyplot as plt
import time
import random

### Formulating the problem

Define number of decision variables, constraints, and specify problem formulation (3- or 5-objective)

In [9]:
# Set the number of decision variables, constraints and performance objectives
nVars = 6   # Define number of decision variables
nObjs = 5   # Define number of objectives
nCnstr = 1      # Define number of decision constraints

# Define the upper and lower bounds of the performance objectives
objs_lower_bounds = [-6000, 0, 0, -250, 0]
objs_upper_bounds = [0, 1, 100, 0, 32000]

### Initialize the problem for optimization
We call the `fisheries_game_problem_setup.py` function to set up the optimization problem. This function returns a PyBorg object called `algorithm` in this exercise that will be optimized in the next step.<br>

In [10]:
# initialize the optimization
algorithm = fisheries_game_problem_setup(nVars, nObjs, nCnstr)

### Define parameters for optimization
Before optimizing, we have to define our desired population size and number of function evaluations (NFEs).

In [16]:
nfe = 10000    # number of function evaluations
pop_size = 500    # population size

### Begin the optimization
In addition to running the optimization, we also time the optimization to get a general estimate on the time the full hypervolume analysis will require.

In [22]:
# open file in which to store optimization objectives and variables
f_objs = open('Fisheries2_objs.txt', "w+")
f_vars = open('Fisheries2_vars.txt', "w+")

# get number of algorithm variables and performance objectives
nvars = algorithm.problem.nvars
nobjs = algorithm.problem.nobjs
    
# begin timing the optimization
opt_start_time = time.time()

algorithm = fisheries_game_problem_setup(nVars, nObjs, nCnstr, pop_size=int(pop_size))
algorithm.run(int(nfe))

# get the solution archive
arch = algorithm.archive[:]
for i in range(len(arch)):
    sol = arch[i]
    # write objectives to file
    for j in range(nobjs):
        f_objs.write(str(sol.objectives[j]) + " ")
    # write variables to file
    for j in range(nvars):
        f_vars.write(str(sol.variables[j]) + " ")
        
    f.write("\n")

# end timing and print optimization time 
opt_end_time = time.time()

opt_total_time = opt_end_time - opt_start_time

f_objs.close()
f_vars.close()

# print the total time to console
print(f"Time taken = ", {opt_total_time})

Time taken =  {2849.4955439567566}


### Output post-processing

First, import the `numpy` library required for output post-processing. Then, convert the text files to csv files:

In [41]:
import numpy as np

# convert txt files to csv 
# load the .txt files as numpy matrices
matrix_objs = np.genfromtxt('Fisheries2_objs.txt', delimiter=' ')
matrix_vars = np.genfromtxt('Fisheries2_vars.txt', delimiter=' ')

# reshape the matrices 
# the objectives file should have shape (n_solns, nObjs)
# the variables file should have shape (n_solns, nVars)
n_solns = int(matrix_objs.shape[0]/nObjs)

matrix_objs = np.reshape(matrix_objs, (n_solns,nObjs))
matrix_vars = np.reshape(matrix_vars, (n_solns,nVars))

# label the objectives and variables
objs_names = ['NPV', 'Pop_Deficit', 'Low_Harvest', 'Worst_Harvest', 'Variance']
var_names = ['c1', 'r1', 'w1', 'c2', 'r2', 'w2']

# Convert the matrices to dataframes with header names
df_objs = pd.DataFrame(matrix_objs, columns=objs_names)
df_vars = pd.DataFrame(matrix_vars, columns=var_names)

# save the processed matrices as csv files
df_objs.to_csv('Fisheries2_objs.csv', sep=',', index=False)
df_vars.to_csv('Fisheries2_vars.csv', sep=',', index=False)